In [6]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoformerConfig, AutoformerModel

from typing import Optional

from utils.time_features import time_features

In [7]:
if torch.backends.mps.is_available():
    DEVICE = 'mps'
elif torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

In [8]:
DATETIME_COLUMN = 'candle_date_time_kst'

In [10]:
class DataUtils:
  
  default_path = os.path.join(Path(os.getcwd()).parent, 'data')
  
  @staticmethod
  def load_parquet(file_name: str, file_dir: Optional[str] = None):
    if not file_dir:
        file_dir = DataUtils.default_path
        
    path = os.path.join(file_dir, file_name)

    if not os.path.exists(path) or file_name.split('.')[-1] != 'parquet':
        return

    print(f'Loading parquet file from: {path}')

    return pd.read_parquet(path)
  

In [143]:
# Custom Dataset for Multivariate Time Series
class TimeSeriesDataset(Dataset):
    
    def __init__(self, features, target, time_features, sequence_length, label_length, prediction_length):
        self.features = features
        self.target = target.flatten()
        # self.time_features = time_features(pd.to_datetime(features[DATETIME_COLUMN].values), freq='H')
        # self.time_features = self._make_time_features(features)
        self.time_features = time_features
        self.sequence_length = sequence_length
        self.label_length = label_length
        self.prediction_length = prediction_length

    def __len__(self):
        return len(self.features) - self.sequence_length - self.prediction_length + 1

    def __getitem__(self, idx):
        start_past = idx
        end_past = idx + self.sequence_length
        start_future = end_past - self.label_length
        end_future = end_past + self.prediction_length
        
        past_values = self.features[start_past : end_past]
        past_time_features = self.time_features[start_past : end_past]
        
        future_values = self.features[start_future : end_future]
        y = self.target[start_future : end_future]
        future_time_features = self.time_features[start_future : end_future]
        # print('&&&& ', self.target.shape, y.shape)
        # y = self.target[idx : idx + self.sequence_length]
        
        past_observed_mask = np.ones((self.sequence_length, past_values.shape[1]))
        future_observed_mask = np.ones((self.sequence_length, future_values.shape[1]))
        
        # print(past_values.shape, past_time_features.shape, future_time_features.shape, past_observed_mask.shape)
        
        return {
            'past_values': torch.tensor(past_values, dtype=torch.float32), 
            'past_time_features': torch.tensor(past_time_features, dtype=torch.float32),
            'past_observed_mask': torch.tensor(past_observed_mask, dtype=torch.bool),
            'future_values': torch.tensor(future_values, dtype=torch.float32), 
            'future_time_features': torch.tensor(future_time_features, dtype=torch.float32),
            'future_observed_mask': torch.tensor(future_observed_mask, dtype=torch.bool),
            'y': torch.tensor(y, dtype=torch.float32),
        }
        
    # def _make_time_features(self, features: pd.DataFrame): 
    #     df_stamp = features[[DATETIME_COLUMN]]
    #     df_stamp['month'] = df_stamp[DATETIME_COLUMN].apply(lambda row: row.month, 1)
    #     df_stamp['day'] = df_stamp[DATETIME_COLUMN].apply(lambda row: row.day, 1)
    #     df_stamp['weekday'] = df_stamp[DATETIME_COLUMN].apply(lambda row: row.weekday(), 1)
    #     df_stamp['hour'] = df_stamp[DATETIME_COLUMN].apply(lambda row: row.hour, 1)
    #     df_stamp['minute'] = df_stamp[DATETIME_COLUMN].apply(lambda row: row.minute, 1)
    #     df_stamp['second'] = df_stamp[DATETIME_COLUMN].apply(lambda row: row.second, 1)
    #     data_stamp = df_stamp.drop([DATETIME_COLUMN], 1).values
    #     return data_stamp

In [144]:
def generate_time_features(data):
    # Extract time-related features from the timestamp column
    timestamps = pd.to_datetime(data[DATETIME_COLUMN])
    time_features = pd.DataFrame({
        "second_sin": np.sin(2 * np.pi * timestamps.dt.second / 24),
        "second_cos": np.cos(2 * np.pi * timestamps.dt.second / 24),
        "minute_sin": np.sin(2 * np.pi * timestamps.dt.minute / 24),
        "minute_cos": np.cos(2 * np.pi * timestamps.dt.minute / 24),
        "hour_sin": np.sin(2 * np.pi * timestamps.dt.hour / 24),
        "hour_cos": np.cos(2 * np.pi * timestamps.dt.hour / 24),
        "day_sin": np.sin(2 * np.pi * timestamps.dt.day / timestamps.dt.days_in_month),
        "day_cos": np.cos(2 * np.pi * timestamps.dt.day / timestamps.dt.days_in_month),
        "month_sin": np.sin(2 * np.pi * timestamps.dt.month / 12),
        "month_cos": np.cos(2 * np.pi * timestamps.dt.month / 12),
    })
    return time_features

def preprocess_data(data, feature_columns, target_column, sequence_length, label_length, prediction_length):
    # Sort by timestamp if necessary
    data = data.sort_values(DATETIME_COLUMN)

    # Normalize the features and target
    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler()

    # features = data.drop(columns=[target_column]).values
    features = data[feature_columns].values
    target = data[target_column].values.reshape(-1, 1)

    features_normalized = scaler_features.fit_transform(features)
    target_normalized = scaler_target.fit_transform(target)
    isnan = np.isnan(features_normalized)
    # print(features_normalized.shape, isnan.shape, features_normalized[~isnan.any(axis=1)].shape, target_normalized[~isnan.any(axis=1)].shape)
    
    # Generate time-based features
    time_features = generate_time_features(data=data).values
    NUM_TIME_FEATURES = time_features.shape[1]
    # print(features.shape, target.shape, time_features.shape)

    # Split into train and validation sets
    
    # Split into train and validation sets
    (
        train_features, 
        val_features, 
        train_target, 
        val_target, 
        train_time_features, 
        val_time_features
    ) = (
        train_test_split(
            features_normalized[~isnan.any(axis=1)], 
            target_normalized[~isnan.any(axis=1)], 
            time_features[~isnan.any(axis=1)], 
            test_size=0.2, 
            shuffle=False
        )
    )
    # train_features, val_features, train_target, val_target = train_test_split(
    #     features_normalized, target_normalized, test_size=0.2, shuffle=False
    # )

    # Create datasets
    train_dataset = TimeSeriesDataset(
        train_features,
        train_target,
        train_time_features,
        sequence_length=sequence_length,
        prediction_length=prediction_length,
        label_length=label_length,
    )
    val_dataset = TimeSeriesDataset(
        val_features,
        val_target,
        val_time_features,
        sequence_length=sequence_length,
        prediction_length=prediction_length,
        label_length=label_length,
    )

    return train_dataset, val_dataset, scaler_features, scaler_target, NUM_TIME_FEATURES

In [145]:
data = DataUtils.load_parquet('IOTA_1s_2000000_2025-01-05T18:19:37+09:00.parquet_20250105181937.parquet')
feature_columns = [
    # 'best_profit_rate', 
    'variance', 
    'worst_profit_rate_before', 
    'opening_price', 
    'high_price', 
    'mid_price', 
    'low_price', 
    'candle_acc_trade_volume', 
    # 'diff_opening_price',
    # 'diff_high_price',
    # 'diff_mid_price',
    # 'diff_low_price', 
    # 'diff_candle_acc_trade_volume',
    'timedelta_after',
]
# dataset = preprocess_data(data, feature_columns, 'best_profit_rate', 60, 10)
  # data[[
  #       'variance', 
  #       # 'best_profit_rate_before',
  #       'worst_profit_rate_before', 
  #       'opening_price', 
  #       'high_price', 
  #       'mid_price', 
  #       'low_price', 
  #       'candle_acc_trade_volume', 
  #       # 'diff_opening_price',
  #       # 'diff_high_price',
  #       # 'diff_mid_price',
  #       # 'diff_low_price', 
  #       # 'diff_candle_acc_trade_volume',
  #       'timedelta_after',
  #     ]],
  #     data[['best_profit_rate']]

Loading parquet file from: /Users/minjiwon/upbase-data-server/data/IOTA_1s_2000000_2025-01-05T18:19:37+09:00.parquet_20250105181937.parquet


In [146]:
TARGET_COLUMN = 'best_profit_rate'

# Parameters
# INPUT_LENGTH = 60  # Number of past time steps to use as input
# OUTPUT_LENGTH = 12  # Number of future time steps to predict
# BATCH_SIZE = 32
# LEARNING_RATE = 5e-4
# EPOCHS = 10

SEQUENCE_LENGTH = 24 * 4 * 4
PREDICTION_LENGTH = 24 * 4
LABEL_LENGTH = 24 * 4

BATCH_SIZE = 128
NUM_BATCHES_PER_EPOCH = 100
EPOCHS = 50
LEARNING_RATE = 5e-4
SCALING = 'std'

In [147]:
# Preprocess data
train_dataset, val_dataset, scaler_features, scaler_target, NUM_TIME_FEATURES = preprocess_data(
  data,
  feature_columns,
  TARGET_COLUMN,
  sequence_length=SEQUENCE_LENGTH,
  prediction_length=PREDICTION_LENGTH,
  label_length=LABEL_LENGTH,
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [148]:
# Load Autoformer Model
configuration = AutoformerConfig(
    input_size=len(feature_columns),  # Number of input features
    prediction_length=PREDICTION_LENGTH,
    context_length=CONTEXT_LENGTH,
    label_length=LABEL_LENGTH,
    # encoder_layers=3,
    # decoder_layers=3,
    # d_model=32,
    lags_sequence=[0],
    num_time_features=NUM_TIME_FEATURES,
    # lags_sequence=list(range(1, 8)),  # Ensure lags are within input_length
)
model = AutoformerModel(configuration)

In [149]:
# Optimizer and Loss Function
device = torch.device(DEVICE)
model.to(device)

# print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.MSELoss()

# Training Loop
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    
    print(f'EPOCH {epoch + 1}')
    
    for batch in train_loader:
        # past_values, future_values, y, time_features, mask = (_d.to(device) for _d in _data)
        def _get(key):
            return batch[key].to(device)
        
        # decoder input
        y = _get('y')
        # dec_inp = torch.zeros_like(y[:, -PREDICTION_LENGTH:, :]).float()
        # dec_inp = torch.cat([y[:, :LABEL_LENGTH, :], dec_inp], dim=1).float().to(device)
        # print('!1', dec_inp.shape, y.shape)
        
        # print(batch['past_values'].shape, batch['future_values'].shape, batch['y'].shape, batch['past_time_features'].shape, batch['future_time_features'].shape, batch['past_observed_mask'].shape)

        optimizer.zero_grad()
        print(_get('past_values').shape, _get('past_time_features').shape, _get('past_observed_mask').shape)
        outputs = model(
            # x.unsqueeze(-1),
            past_values=_get('past_values'),
            past_time_features=_get('past_time_features'),  # Placeholder for time-based features
            past_observed_mask=_get('past_observed_mask'),  # Mask indicating observed data
            future_values=_get('future_values'),
            future_time_features=_get('future_time_features'), 
            # future_observed_mask=_get('future_observed_mask'),
            return_dict=True,
        )
        print(outputs.keys(), y.shape, y)
        for key in ['last_hidden_state', 'trend', 'encoder_last_hidden_state', 'loc', 'scale', 'static_features']:
        # for key in ['encoder_last_hidden_state', 'loc', 'scale', 'static_features']:
            print(key, outputs[key].shape if isinstance(outputs[key], torch.Tensor)  else outputs[key])
            
        print(outputs.trend)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0.0
    
    with torch.no_grad():
        for batch in train_loader:
            def _get(key):
                return batch[key].to(device)
            
            outputs = model(
                # x.unsqueeze(-1),
                past_values=_get('past_values'),
                past_time_features=_get('past_time_features'),  # Placeholder for time-based features
                future_values=_get('future_values'),
                future_time_features=_get('future_time_features'), 
                past_observed_mask=_get('past_observed_mask'),  # Mask indicating observed data
            )
            loss = criterion(outputs, _get('y'))
            val_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Train Loss: {train_loss / len(train_loader):.4f}, Val Loss: {val_loss / len(val_loader):.4f}")

EPOCH 1
torch.Size([128, 384, 8]) torch.Size([128, 384, 10]) torch.Size([128, 384, 8])


ValueError: input length 480 and time feature lengths 576 does not match

In [ ]:
# Save the model
model.save_pretrained('./autoformer_model')